<a href="https://colab.research.google.com/github/YanNews2805/CalculoNumerico/blob/main/Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gmail webscrapping automation**

##**Requirements**

* **Make sure you enable IMAP in your Gmail settings**


1.   *Log on to your Gmail account;*
2.   *Settings*
3.   *See All Settings*
4.   *Select Forwarding and POP/IMAP tab*
5.   *In the "IMAP access” section, select Enable IMAP*


* **If you have 2-factor authentication, gmail requires you to create an application specific password that you need to use.**

1. *Go to your Google account settings and click on 'Security'*
2. *Scroll down to App Passwords under 2 step verification*
3. *Select Mail under Select App. and Other under Select Device*
4. *The system gives you a password that you need to use to authenticate from python*


##**Input**

*Insert the date you want to get the emails*

**Example of input:** *'09-Jun-2024'*

In [ ]:
date_to_filter = "20-Jun-2024"

##**Importing libraries**

In [ ]:
import imaplib
from email.policy import default
import email
import re
from datetime import datetime
import subprocess
import requests
import json
import pandas as pd
import base64
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup

## **Functions**

###**Setup connections**

In [ ]:
def setup_connections(user, password, imap_url='imap.gmail.com'):
    """
    Establishes a connection to the email server using IMAP and logs in, then retrieves the API key.

    Parameters:
    user (str): Email address for login.
    password (str): Password for login.
    imap_url (str): URL for the IMAP server. Default is 'imap.gmail.com'.

    Returns:
    tuple: A tuple containing the IMAP connection object and the API key.
    """
    try:
        # Connection with the email server using SSL
        my_mail = imaplib.IMAP4_SSL(imap_url)
        # Log in using the provided credentials
        my_mail.login(user, password)
        # Select the Inbox to fetch messages
        my_mail.select('Inbox')
        print("Connected to email successfully.")

        # Retrieve the API key
        api_key = 'API key'  # Replace with secure method of retrieving the API key
        return my_mail, api_key
    except Exception as e:
        print(f"An error occurred while setting up connections: {e}")
        return None, None

### **Formatted date**

In [ ]:
def format_email_date(response_part):
    """
    Formats the date from an email response part to a specific format.

    Args:
    response_part (tuple): A tuple containing parts of the email response,
                           where response_part[1] is the email message bytes.

    Returns:
    str: The formatted email date in 'dd-MMM-YYYY' format.
    """
    # Parse the email message from the response part bytes
    my_msg = email.message_from_bytes(response_part[1])

    # Extract the 'Date' header from the email message
    email_date = my_msg['Date']

    # Limit the length of the date string to handle different formats
    email_date = email_date[0:25]

    # Check if the date string needs reformatting for single-digit days
    if email_date[5] in ['0', '1', '2', '3'] and email_date[6] != ' ':
        # Format the date string if no reformatting is needed
        email_date_formatted = datetime.strptime(email_date, '%a, %d %b %Y %H:%M:%S').strftime('%d-%b-%Y')
    else:
        # Reformat the date string by adding a leading zero to the day
        email_date = email_date[0:4] + ' 0' + email_date[5:24]
        email_date_formatted = datetime.strptime(email_date, '%a, %d %b %Y %H:%M:%S').strftime('%d-%b-%Y')

    return email_date_formatted


### **Fetch emails**

In [ ]:
def fetch_emails(my_mail, date_to_filter):
    """
    Searches for emails in the inbox from a specific date and fetches them.

    Parameters:
    my_mail (imaplib.IMAP4_SSL): The IMAP connection object.
    date_to_filter (str): The specific date to filter emails (format: 'DD-Mon-YYYY').

    Returns:
    list: A list of fetched email data.
    """
    # Search for emails from the specified date
    search_criterion = f'(ON {date_to_filter})'
    status, data = my_mail.search(None, search_criterion)

    # Check if the search was successful
    if status != 'OK':
        print("Error in searching emails.")
        exit()

    # Get the list of email IDs
    mail_id_list = data[0].split()  # IDs of all emails that we want to fetch

    msgs = []  # empty list to capture all messages

    # Iterate through the messages and extract data into the messages list
    for num in mail_id_list:
        # Ensure the email ID is properly formatted and decoded
        if isinstance(num, bytes):
            num = num.decode('utf-8')

        # Fetch the email data
        typ, data = my_mail.fetch(num, '(RFC822)')  # RFC822 returns the whole message

        if typ != 'OK':
            print(f"Error fetching email ID {num}")
            continue

        msgs.append(data)

    return msgs

### **Extract email info and company category**

In [ ]:
def extract_email_info(sender_email, my_msg):
    """
    Extracts deal type, organization name, and body text from an email message.

    Args:
    sender_email (str): The email address of the sender.
    my_msg (object): The email message object to extract information from.

    Returns:
    tuple: A tuple containing deal_type (str), organization_name (str), and body_text (str or None).
    """
    deal_type = None
    organization_name = None
    body_text = None

    # Extract domain name from sender's email
    domain_match = re.search(r'@([^.]+)', sender_email)
    extracted_name = domain_match.group(1) if domain_match else None


    # Extract text from email body
    for part in my_msg.walk():

        if part.get_content_type() == 'text/plain':
            if part.get('Content-Transfer-Encoding') == 'base64':
                body_text = base64.b64decode(part.get_payload()).decode('utf-8')
            else:
                body_text = part.get_payload()

            break  # Assuming we only care about the first plain text part

    # Extract the subject of the email
    subject_text = my_msg['Subject']


    if body_text:
        # Check for transaction type using regular expressions
        keywords = ['pass(?:ed|ing)?', 'FYI', 'For your information', 'not a fit', '46YI', 'does not fit']
        already_in_affinity = [
            'Graphene Ventures - Introduction',
            'Re: Graphene Ventures - Introduction',
            'Graphene Ventures - Follow up',
            'Re: Graphene Ventures - Follow up',
            'Re: Graphene Ventures - Intro Call Follow Up',
            'Graphene Ventures - First Call',
            'Graphene Ventures - Follow Up'
        ]

        # Identify if email is already in affinity based on subject or body content
        if subject_text in already_in_affinity or any(
                link in body_text for link in [
                    'https://calendly.com/arthur-grapheneventures/founder-meeting',
                    'https://calendly.com/leticiapamplona/45-minute-meeting',
                    'https://calendly.com/jorge-grapheneventures/30min-foundermeeting']):
            deal_type = 'Already in Affinity'
        else:
            deal_type = 'Analyze'


    # Assign organization name based on the extracted domain
    organization_name = sender_email if extracted_name == 'gmail' else extracted_name


    return deal_type, organization_name, body_text

### **Extract company name**

In [ ]:
def extract_company_name(email_ext):
    """
    Extracts the company name from an email address.

    Args:
    email_ext (str): The email address to extract the company name from.

    Returns:
    str: The extracted company name.
    """

    match = re.search(r"@([^@]+)\.com", email_ext)
    return match.group(1) if match else None

### **Get Introduced name**

In [ ]:
def extract_introduced_name(email_body):
    """
    Extracts the name introduced in an email body following the phrase 'introduce you to'.

    Args:
    email_body (str): The body of the email from which to extract the name.

    Returns:
    str: The name introduced after the phrase 'introduce you to', or None if no match is found.
    """
    # Define the pattern to search for the phrase 'introduce you to' followed by the name
    pattern = r'introduce you to\s+(\w+(?:\s+\w+)*)'

    # Search for the pattern in the email body, ignoring case
    match = re.search(pattern, email_body, re.IGNORECASE)

    if match:
        # Extract and return the introduced name if a match is found
        introduced_name = match.group(1)
        return introduced_name
    else:
        # Return None if no match is found
        return None


### **Get emails in CC**

In [ ]:
def get_cc_with_different_email(msg, sender_email):
    """
    Extracts CC email addresses from a message that are different from the sender's email
    and do not contain 'graphenevc.com'.

    Args:
    msg (object): The email message object from which to extract CC addresses.
    sender_email (str): The sender's email address to compare against.

    Returns:
    str: The first CC email address that meets the criteria, or None if no such address is found.
    """
    # Extract CC email addresses from the message
    cc_list = msg.get_all('cc', [])

    # Iterate through the CC addresses to find one that meets the criteria
    for cc in cc_list:
        cc_email = cc.strip()
        if cc_email != sender_email and 'graphenevc.com' not in cc_email:
            return cc_email

    # Return None if no address meets the criteria
    return None

#### **Handle CC list**

In [ ]:
def handle_cc_list(cc_list, body_text, my_msg, sender_email, sent_to):
    """
    Handles emails with CC list.

    Args:
    cc_list (list): The list of CC email addresses.
    body_text (str): The body text of the email.
    my_msg (object): The email message object to handle.
    sender_email (str): The email address of the sender.
    sent_to (str): The email address of the recipient.
    """

    for cc_email in cc_list:
        if '@graphenevc.com' in cc_email:
            extracted_name = extract_company_name(sent_to)
        else:
            introduced_name = extract_introduced_name(body_text)
            extracted_name = introduced_name if introduced_name else extract_company_name(cc_email)


#### **Handle no CC list**

In [ ]:
def handle_no_cc(my_msg, sender_email, sent_to):
    """
    Handles emails with no CC list.

    Args:
    my_msg (object): The email message object to handle.
    sender_email (str): The email address of the sender.
    sent_to (str): The email address of the recipient.
    """

    info = extract_email_info(sent_to, my_msg)

    new_row_data = (info[0], extract_email_info(sent_to, my_msg)[1], remove_signature(info[2]))
    add_to_dataframe(new_row_data)

### **Get emails in body**

In [ ]:
def extract_body_email(my_msg):
    """
    Extracts email addresses from the plain text body of an email message.

    Args:
    my_msg (object): The email message object to extract email addresses from.

    Returns:
    str: The extracted email address or domain, or None if no matching email is found.
    """
    extracted_email = None

    # Walk through the email parts
    for part in my_msg.walk():
        # Check if the part is plain text
        if part.get_content_type() == 'text/plain':
            # Decode the part if it is base64 encoded
            if part.get('Content-Transfer-Encoding') == 'base64':
                body_text = base64.b64decode(part.get_payload()).decode('utf-8')
            else:
                body_text = part.get_payload()

            # Extract email addresses from the body text
            emails_in_body = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', body_text)

            for email_addr in emails_in_body:
                # Extract the domain name from the email address
                domain_match = re.search(r'@([^.]+)', email_addr)
                if domain_match:
                    domain_name = domain_match.group(1)
                else:
                    continue

                # Check if the domain name is not 'nmrk' or 'graphenevc'
                if domain_name not in {'nmrk', 'graphenevc'}:
                    # Return the whole email address if domain is 'gmail', 'openvc', 'ligopartners', or 'easyintro'
                    if domain_name in {'gmail', 'openvc', 'ligopartners', 'easyintro'}:
                        return email_addr
                    else:
                        # Return the extracted domain name if different
                        return email_addr.split('@')[1].split('.')[0]

    # Return None if no matching email address is found
    return extracted_email

### **Remove signature**

In [ ]:
def remove_signature(email_body):
    """
    Removes signature blocks from the email body.

    Args:
    email_body (str): The body of the email.

    Returns:
    str: The cleaned email body without signature blocks.
    """
    # Define patterns to identify signature blocks
    signature_patterns = [
        r'(?i)(This message may contain privileged or confidential information.*)'  # Confidentiality notice
    ]

    # Combine all patterns into one regex
    combined_pattern = re.compile('|'.join(signature_patterns), re.MULTILINE | re.DOTALL)

    # Replace all matched patterns with an empty string
    cleaned_email_body = combined_pattern.sub('', email_body)

    # Return the cleaned email body
    return cleaned_email_body.strip()

###**Define how to handle the email**

In [ ]:
def process_email(my_msg, sender_email, sent_to):
    """
    Processes an email based on the sender's email and the recipient's email.

    Args:
    my_msg (object): The email message object to process.
    sender_email (str): The email address of the sender.
    sent_to (str): The email address of the recipient.
    """

    if 'vc' in sender_email or sender_email == 'info@coordinator.com' or 'partners' in sender_email or 'nmrk' in sender_email:
        handle_graphene_email(my_msg, sender_email, sent_to)

    else:

        handle_outside_email(my_msg, sender_email)

####**Handle Graphene Emails**

In [ ]:
def handle_graphene_email(my_msg, sender_email, sent_to):
    """
    Handles emails related to Graphene Ventures.

    Args:
    my_msg (object): The email message object to handle.
    sender_email (str): The email address of the sender.
    sent_to (str): The email address of the recipient.
    """

    if 'graphenevc' in sent_to:
        process_internal_graphene_email(my_msg, sender_email)
    else:
        process_external_graphene_email(my_msg, sender_email, sent_to)

#####**Process Internal Graphene Email**

In [ ]:
def process_internal_graphene_email(my_msg, sender_email):
    """
    Processes internal emails within Graphene Ventures.

    Args:
    my_msg (object): The email message object to process.
    sender_email (str): The email address of the sender.
    """

    reply_email = my_msg['reply-to']
    if reply_email:

        add_to_dataframe(extract_email_info(reply_email, my_msg))
    else:

        info = extract_email_info(sender_email, my_msg)

        new_row_data = (info[0], extract_body_email(my_msg), info[2])
        add_to_dataframe(new_row_data)

#####**Process External Graphene Email**

In [ ]:
def process_external_graphene_email(my_msg, sender_email, sent_to):
    """
    Processes external emails involving Graphene Ventures.

    Args:
    my_msg (object): The email message object to process.
    sender_email (str): The email address of the sender.
    sent_to (str): The email address of the recipient.
    """

    info = extract_email_info(sent_to, my_msg)
    cc_list = get_cc_with_different_email(my_msg, sender_email)

    if cc_list:
        handle_cc_list(cc_list, info[2], my_msg, sender_email, sent_to)
    else:
        handle_no_cc(my_msg, sender_email, sent_to)

####**Handle Outside Email**

In [ ]:
def handle_outside_email(my_msg, sender_email):
    """
    Handles emails sent from outside Graphene Ventures.

    Args:
    my_msg (object): The email message object to handle.
    sender_email (str): The email address of the sender.
    """

    info = extract_email_info(sender_email, my_msg)
    new_row_data = (info[0], info[1], remove_signature(info[2]))
    add_to_dataframe(new_row_data)


### **Add to dataframe**

In [ ]:
def add_to_dataframe(data):
    """
    Adds a new row of data to the DataFrame.

    Args:
    data (tuple): The data to add as a new row.
    """
    global df
    new_row_series = pd.Series(data, index=df.columns)
    df = pd.concat([df, new_row_series.to_frame().T], ignore_index=True)


###**Affinity**

####**Create organization**

In [ ]:
def create_organization(organization_name, api_key):
    """
    Creates an organization in Affinity.

    Args:
        organization_name (str): The name of the organization to create.
        api_key (str): The API key for authentication.

    Returns:
        str: The ID of the created organization.
    """
    url_org = 'https://api.affinity.co/organizations'
    headers_org = {'Content-Type': 'application/json'}
    data_org = {'name': organization_name}
    response_org = requests.post(url_org, headers=headers_org, json=data_org, auth=('', api_key))
    organization_data = response_org.json()
    return organization_data.get('id')

####**Create notes**

In [ ]:
def create_note(organization_id, note_content, api_key):
    """
    Creates a note for an organization in Affinity.

    Args:
        organization_id (str): The ID of the organization.
        note_content (str): The content of the note.
        api_key (str): The API key for authentication.
    """
    url_note = "https://api.affinity.co/notes"
    headers = {"Content-Type": "application/json"}
    data = {"organization_ids": [organization_id], "content": note_content}
    response = requests.post(url_note, auth=('', api_key), headers=headers, json=data)
    return response

####**Add to Affinity list**

In [ ]:
def add_to_list(organization_id, api_key):
    """
    Adds an organization to a list in Affinity.

    Args:
        organization_id (str): The ID of the organization.
        api_key (str): The API key for authentication.
    """
    url_list = "https://api.affinity.co/lists/108607/list-entries"
    headers_list = {'Content-Type': 'application/json'}
    data_list = {
        'entity_id': organization_id,
        "value_type": 1,
        "list_id": LIST,
        "name": "Official Deal Flow + Portfolio",
        "is_public": True
    }
    response = requests.post(url_list, json=data_list, auth=('', api_key))
    return response

####**Build a list with all the Affinity companies**

In [ ]:
# Constants
api_url = 'https://api.affinity.co/lists/LIST/list-entries'
headers = {'Content-Type': 'application/json'}
params = {'list_id': LIST}

def fetch_affinity_companies(api_url, headers, params, api_key):
    """
    Fetches companies from the Affinity API.

    :param api_url: The URL for the API request
    :param headers: Headers for the API request
    :param params: Parameters for the API request
    :param api_key: API key for authentication
    :return: List of company names
    """

    response = requests.get(api_url, headers=headers, params=params, auth=('', api_key))
    data = response.json()
    companies = []
    for company in data:
      x = company['entity']['name']
      companies.append(x)
    return companies

####**Process organizations to add in Affinity**

In [ ]:
def process_organizations(organization_names, companies, df, api_key):
    """
    Processes a list of organization names, creating organizations and notes in Affinity if they are not already in the companies list.

    Parameters:
    organization_names (list): List of organization names to process.
    companies (list): List of existing companies to check against.
    df (pd.DataFrame): DataFrame containing data related to the organizations.
    api_key (str): API key for accessing the Affinity API.

    Returns:
    pd.DataFrame: Updated DataFrame with status of organizations being added to Affinity.
    """
    count = 0  # Initialize a counter

    for organization_name in organization_names:
        stripped_name = organization_name.strip()
        if organization_name and stripped_name not in companies and organization_name != []:
            companies.append(organization_name)
            try:
                # Create the organization in Affinity
                organization_id = create_organization(organization_name, api_key)
                # Create a note for the organization
                create_note(organization_id, df.iloc[count, 2], api_key)
                # Add the organization to a list in Affinity
                add_to_list(organization_id, api_key)
                df.at[count, 'Going to affinity'] = 'Yes'
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                count += 1
        else:
            df.at[count, 'Going to affinity'] = 'No, already in dealflow'
            count += 1

    return df

###**Main function**

In [ ]:
def main_function(my_mail, date_to_filter):
    """
    Processes the emails by fetching them and filtering based on the given date.

    Parameters:
    my_mail (imaplib.IMAP4_SSL): The IMAP connection object.
    date_to_filter (str): The specific date to filter emails.

    Returns:
    pd.DataFrame: The DataFrame after processing and filtering emails.
    """

    # Fetch the emails
    msgs = fetch_emails(my_mail,date_to_filter)
    global df
    df = pd.DataFrame(columns=['Category', 'Company name', 'Text'])

    # Logout and close the connection
    my_mail.logout()
    # Main processing loop
    for msg in msgs[::-1]:
        for response_part in msg:
            if isinstance(response_part, tuple):
                my_msg = BytesParser(policy=policy.default).parsebytes(response_part[1])

                email_date_formatted = format_email_date(response_part)


                if email_date_formatted == date_to_filter:
                    sent_to = my_msg['to']
                    sender_email = my_msg['from']

                    process_email(my_msg, sender_email, sent_to)

    # Drop rows with missing 'Text' values
    df = df.dropna(subset=['Text'])
    return df

##**Main**

In [ ]:
user, password = USER, PASSWROD
my_mail, api_key = setup_connections(user, password)
df = main_function(my_mail, date_to_filter)
organization_names = df['Company name']
companies = fetch_affinity_companies(api_url, headers, params, api_key)
updated_df = process_organizations(organization_names, companies, df, api_key)
updated_df

Connected to email successfully.


,Category,Company name,Text,Going to affinity
0,Analyze,neto,"Gabe,\r\n\r\nThanks so much.\r\n\r\nProject Su...",Yes
